In [0]:
#Delta Time Travel (versions)

tbl = "workspace.default.gold_monthly_revenue"
spark.sql(f"DESCRIBE HISTORY {tbl}").show(truncate=False)
display(spark.sql(f"SELECT * FROM {tbl} VERSION AS OF 0"))


In [0]:
#Schema enforcement vs evolution

#df1= spark.createDataFrame([(1,"A"),(2,"B")],["id","name"])
#df1.write.format("delta").mode("overwrite").saveAsTable("workspace.default.delta_schema_demo")
#display(spark.table("workspace.default.delta_schema_demo"))

#df2= spark.createDataFrame([(3,"C"),(4,"D")],["id","name"])
#df2.write.format("delta").mode("append").saveAsTable("workspace.default.delta_schema_demo")
#display(spark.table("workspace.default.delta_schema_demo"))

df2.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("workspace.default.delta_schema_demo")

display(spark.table("workspace.default.delta_schema_demo"))

df3= spark.createDataFrame([(3,"C"),(4,"D")],["id","name"])
df3.write.format("delta").mode("append").saveAsTable("workspace.default.delta_schema_demo")
display(spark.table("workspace.default.delta_schema_demo"))





In [0]:
# small incremental updates
incr = spark.createDataFrame([(1, 107, 999.0)], ["CustomerId","OrderId","Amount"])
incr.createOrReplaceTempView("src")

spark.sql("""
MERGE INTO workspace.default.silver_orders AS tgt
USING src
ON tgt.CustomerId = src.CustomerId AND tgt.OrderId = src.OrderId
WHEN MATCHED THEN UPDATE SET tgt.Amount = src.Amount
""")

display(spark.sql("SELECT * FROM workspace.default.silver_orders WHERE CustomerId=1 AND OrderId=107"))

In [0]:
spark.sql("describe history workspace.default.delta_schema_demo").show()

In [0]:
display(spark.table("workspace.default.delta_schema_demo"))

In [0]:
%sql
OPTIMIZE workspace.default.silver_orders ZORDER BY (CustomerId);